In [7]:
import requests
from itertools import chain
from collections import namedtuple
import logging

from lxml import etree
from pprint import pprint
from IPython.core.display import display, HTML

from io import StringIO

In [8]:
with open('src/battlescribe/catelogue_schema.xsd', 'rb') as fh:
    catelogue_schema = etree.XMLSchema(etree.fromstring(fh.read()))

In [9]:
path = 'https://raw.githubusercontent.com/BSData/wh40k/master/{}'.format('Warhammer 40,000.gst')
r = requests.get(path)
base = etree.fromstring(r.content, etree.XMLParser(remove_blank_text=True, schema = None))

In [10]:
for x in base:
    print(x.tag, len(x))

{http://www.battlescribe.net/schema/gameSystemSchema}readme 0
{http://www.battlescribe.net/schema/gameSystemSchema}publications 28
{http://www.battlescribe.net/schema/gameSystemSchema}costTypes 3
{http://www.battlescribe.net/schema/gameSystemSchema}profileTypes 10
{http://www.battlescribe.net/schema/gameSystemSchema}categoryEntries 68
{http://www.battlescribe.net/schema/gameSystemSchema}forceEntries 13
{http://www.battlescribe.net/schema/gameSystemSchema}entryLinks 20
{http://www.battlescribe.net/schema/gameSystemSchema}sharedSelectionEntries 181
{http://www.battlescribe.net/schema/gameSystemSchema}sharedSelectionEntryGroups 3
{http://www.battlescribe.net/schema/gameSystemSchema}sharedRules 17
{http://www.battlescribe.net/schema/gameSystemSchema}sharedProfiles 190


In [11]:
set(y.attrib['typeName'] for y in
    base.xpath(
        "gc:sharedProfiles/gc:profile", 
        namespaces={'gc': 'http://www.battlescribe.net/schema/gameSystemSchema'}))

{'Abilities', 'Psychic Power', 'Unit', 'Weapon'}

In [20]:
for y in base.xpath(
    "//gc:entryLinks", 
    namespaces={'gc': 'http://www.battlescribe.net/schema/gameSystemSchema'})[:1]:
    print(y.tag, y.attrib)
    for x in y:
        print(x.tag, x.attrib)
        print(etree.tostring(x))
        print()

{http://www.battlescribe.net/schema/gameSystemSchema}entryLinks {}
{http://www.battlescribe.net/schema/gameSystemSchema}entryLink {'id': '46df-b154-20ff-cce1', 'name': 'Aegis Defence Line', 'hidden': 'false', 'collective': 'false', 'import': 'true', 'targetId': 'a505-05af-bd44-56b6', 'type': 'selectionEntry'}
b'<entryLink xmlns="http://www.battlescribe.net/schema/gameSystemSchema" id="46df-b154-20ff-cce1" name="Aegis Defence Line" hidden="false" collective="false" import="true" targetId="a505-05af-bd44-56b6" type="selectionEntry"><categoryLinks><categoryLink id="46df-b154-20ff-cce1-d713cda3-5d0f-40d8-b621-69233263ec2a" hidden="false" targetId="d713cda3-5d0f-40d8-b621-69233263ec2a" primary="true"/></categoryLinks></entryLink>'

{http://www.battlescribe.net/schema/gameSystemSchema}entryLink {'id': '8117-a765-8d79-07a8', 'name': 'Aquila Strongpoint', 'hidden': 'false', 'collective': 'false', 'import': 'true', 'targetId': '16d6-25c4-af92-4329', 'type': 'selectionEntry'}
b'<entryLink xmlns=

In [13]:
# remove duplicates within selection
# group units weapons and abilities tables

In [28]:
with open("src/battlescribe/catelogue.xslt", "r") as f:
    xslt_root = etree.XML(f.read())
transform = etree.XSLT(xslt_root)
result_tree = transform(base)
with open("src/battlescribe/test.html", "w") as f:
    f.write(str(result_tree))